In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
import random

import sklearn

In [ ]:
pd.__version__

'2.2.2'

In [ ]:
sklearn.__version__

'1.6.1'

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.1 MB/s eta 0:00:00


In [ ]:
users=pd.read_csv("/content/subset_users (1).csv")
destinations=pd.read_csv("/content/subset_places (1).csv")
reviews=pd.read_csv('/content/subset_reviews (1).csv')

In [ ]:
users

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,LbPkYMGmsu4snHkY-Ri1ww,David,14,2011-03-05 05:28:00,10,3,2,NaN,"opIM6lGvGNIVL8tMDa3_Uw, 5wqafYyVJM5lKbOMIs-A3Q...",2,...,0,0,0,0,3,1,0,0,0,0
1,K7thO1n-vZ9PFYiC7nTR2w,Yelper,1554,2007-12-26 23:05:41,11276,6580,8918,"2008,2011,2012,2013,2014","dLOfp-2TAGAb78jGZPvMoA, HQF0jsricHg6n35DXjvh3w...",326,...,182,108,69,58,1259,2765,1900,1900,844,105
2,w-zD8Ln3XZszM82AfVrspg,Robin,90,2008-05-05 02:55:07,137,36,66,2010,"RgDVC3ZUBqpEe6Y1kPhIpw, -Yuf4Ph_6DzKwDOudoRc_A...",5,...,2,0,1,0,4,5,9,9,3,1
3,xwLpDq-rYnyObtBZCD0Wgg,Ellie,191,2009-02-26 13:42:02,281,71,70,"2017,2018,2019,20,20,2021","y8aWXOimQ9ZgUgZ6q--nCQ, 4CC8PtCLBnp26KCVY0vuwA...",7,...,2,0,0,0,9,14,4,4,4,1
4,CsLsfpHlHrInwBfJmjR-zg,Corazon,321,2006-12-05 22:05:19,1819,1300,1678,"2008,2009,2010,2011","Pq-XQyj1fR17v3TJqHKlmA, x6sdOiO42qyUPogtnDLv6A...",90,...,71,62,100,27,218,249,570,570,131,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509,qde7cw7YIJutTmRiFTEmfg,Saikat,20,2010-06-16 02:07:53,70,3,7,NaN,NaN,0,...,0,0,0,0,1,0,0,0,0,0
4510,J1BZQV3C-CBunLi48n31LA,Brian,25,2018-04-15 23:13:41,3,1,0,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,0
4511,jiJDLfLBFgT08H0k7zBj9w,Bob,9,2011-06-08 06:02:59,7,3,2,NaN,NaN,0,...,0,0,0,0,0,0,1,1,0,0
4512,C6jbdotgp_a7ATQ2qhomZw,Debbie,18,2013-01-02 00:59:01,8,3,0,NaN,NaN,0,...,0,0,0,0,2,0,0,0,0,0


In [ ]:
destinations

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,0,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,1,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
2,7clCBgNbd-x2Wj96lZ6Mjw,Bier Brewery and Tap Room,5133 E 65th St,Indianapolis,IN,46220,39.875532,-86.082857,4.5,139,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Beer, Wine & Spirits, Breweries","{'Tuesday': '15:0-21:0', 'Wednesday': '15:0-21..."
3,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,PA,19104,39.954573,-75.194894,3.0,56,1,"{'Alcohol': ""u'none'"", 'RestaurantsGoodForGrou...","Restaurants, Automotive, Delis, Gas Stations, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
4,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,1,"{'RestaurantsTakeOut': 'True', 'NoiseLevel': ""...","Food, Restaurants, Salad, Coffee & Tea, Breakf...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-14:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Zi-F-YvyVOK0k5QD7lrLOg,Muriel's Jackson Square,801 Chartres St,New Orleans,LA,70116,29.958458,-90.063188,4.0,1963,1,"{'RestaurantsGoodForGroups': 'True', 'NoiseLev...","Cajun/Creole, Restaurants","{'Monday': '15:0-21:30', 'Tuesday': '15:0-21:3..."
1074,L62NXZYNN0A2dWZvCj0RkQ,Devlin Collision Center,817 Aramingo Ave,Philadelphia,PA,19125,39.973232,-75.123753,5.0,31,1,{'BusinessAcceptsCreditCards': 'True'},"Automotive, Body Shops",NaN
1075,uyQGW7kZKk90vEwGo8A7mA,Courtesy Hyundai Tampa,3810 W Hillsborough Ave,Tampa,FL,33614,27.995070,-82.507258,3.0,178,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Auto Parts & Supplies, Auto Repair, Car Dealer...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."
1076,Oun4NN-u5yiHIxDqtJnxgA,Federal Donuts,1219 S 2nd St,Philadelphia,PA,19147,39.932223,-75.147463,4.0,530,1,"{'RestaurantsGoodForGroups': 'False', 'GoodFor...","Restaurants, Korean, American (Traditional), B...","{'Monday': '8:0-15:0', 'Tuesday': '8:0-15:0', ..."


In [ ]:
reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
4725,f3YTZqVfGrBlMmfKA9HaSA,FQe-tDY2ESxrvgIyMyKQRg,dz_aIFbATP2PLWQSOBnMfw,3,1,0,0,Nothing special. Came for brunch and got the b...,2014-11-12 17:51:40
4726,SVPfuieZd-Uz9OVkHR3thw,AhqX56lEpj7MjW3UGF9zsQ,_aKr7POnacW_VizRKBpCiA,5,0,0,0,Love this place! Been going back once a week ...,2015-07-03 22:50:05
4727,C6aQ4bkgOGRQLFPu3tRutw,pg0GI_LBpsX06a8eN7Ff5A,0qu0fNTOsSmuREYVIMPuIQ,4,2,0,0,I went here 5-7 years ago. I remember it bein...,2012-07-16 00:18:54
4728,UIkEO-10J6Y99IhRqUflvg,lYAmgL_l7A3MPFYe1DYKrw,EpREWeEpmR8f1qLHzzF0AA,5,0,1,0,Just about to get tucked into a meatloaf that ...,2018-01-09 20:26:13


In [ ]:
def one_hot_encode_categories(df, column_name):
    """
    Perform one-hot encoding on a specified column containing comma-separated categories.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing comma-separated categories.

    Returns:
        pd.DataFrame: A DataFrame with one-hot encoded columns for each unique category.
    """
    # Step 1: Extract unique categories
    all_categories = df[column_name].str.split(',').explode().str.strip()
    unique_categories = all_categories.unique()

    # Step 2: Create one-hot encoding for each category
    for category in unique_categories:
        df[category] = df[column_name].apply(lambda x: 1 if category in x else 0)

    return df

In [ ]:
destinations=one_hot_encode_categories(destinations,"categories")

<ipython-input-10-af4801bd6706>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[category] = df[column_name].apply(lambda x: 1 if category in x else 0)
<ipython-input-10-af4801bd6706>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[category] = df[column_name].apply(lambda x: 1 if category in x else 0)
<ipython-input-10-af4801bd6706>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once us

In [ ]:
destinations

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Cooking Schools,Karaoke,Pediatricians,Service Stations,Transmission Repair,Pet Training,Electricians,Waffles,Basque,Bridal
0,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,AB,T5J 1B9,53.546045,-113.499169,4.0,40,...,0,0,0,0,0,0,0,0,0,0
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,...,0,0,0,0,0,0,0,0,0,0
2,7clCBgNbd-x2Wj96lZ6Mjw,Bier Brewery and Tap Room,5133 E 65th St,Indianapolis,IN,46220,39.875532,-86.082857,4.5,139,...,0,0,0,0,0,0,0,0,0,0
3,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,PA,19104,39.954573,-75.194894,3.0,56,...,0,0,0,0,0,0,0,0,0,0
4,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Zi-F-YvyVOK0k5QD7lrLOg,Muriel's Jackson Square,801 Chartres St,New Orleans,LA,70116,29.958458,-90.063188,4.0,1963,...,0,0,0,0,0,0,0,0,0,0
1074,L62NXZYNN0A2dWZvCj0RkQ,Devlin Collision Center,817 Aramingo Ave,Philadelphia,PA,19125,39.973232,-75.123753,5.0,31,...,0,0,0,0,0,0,0,0,0,0
1075,uyQGW7kZKk90vEwGo8A7mA,Courtesy Hyundai Tampa,3810 W Hillsborough Ave,Tampa,FL,33614,27.995070,-82.507258,3.0,178,...,0,0,0,0,0,0,0,0,0,0
1076,Oun4NN-u5yiHIxDqtJnxgA,Federal Donuts,1219 S 2nd St,Philadelphia,PA,19147,39.932223,-75.147463,4.0,530,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def delete_unused_columns(df,delete_cols):
  cleaned=df.drop(columns=delete_cols)
  return cleaned


In [ ]:
destinations_cleaned=delete_unused_columns(destinations, [
    'name',
    'address','city','state',"postal_code"  , "hours","categories","attributes" ,"latitude","longitude"  ])

In [ ]:
users_cleaned=delete_unused_columns(users, [ 'name', "friends"  , 'elite' ,"yelping_since" ])

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# List of columns to exclude from encoding
exclude_columns_destinations = ['business_id']  # Example for destinations
exclude_columns_users = ['user_id', 'friends']  # Example for users

# Function to encode categorical columns and normalize numeric columns
def encode_categorical(df, exclude_columns, normalize=True):
    # Identify categorical columns (usually object type)
    categorical_columns = df.select_dtypes(include=['object']).columns

    # Exclude the columns you don't want to encode
    categorical_columns = [col for col in categorical_columns if col not in exclude_columns]

    # Apply one-hot encoding for categorical columns
    df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

    # Convert boolean columns (if any) to numerical 0 and 1
    bool_columns = df_encoded.select_dtypes(include=[bool]).columns
    df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

    # Normalize numeric columns if needed
    if normalize:
        numeric_columns = df_encoded.select_dtypes(include=['number']).columns
        scaler = StandardScaler()  # You can also use MinMaxScaler()
        df_encoded[numeric_columns] = scaler.fit_transform(df_encoded[numeric_columns])

    return df_encoded

# Encode categorical columns for both DataFrames excluding specified columns
destinations_encoded = encode_categorical(destinations_cleaned, exclude_columns_destinations)
users_encoded = encode_categorical(users_cleaned, exclude_columns_users)


In [ ]:
destinations_encoded

,business_id,stars,review_count,is_open,Coffee & Tea,Food,Cafes,Bars,Wine Bars,Restaurants,...,Cooking Schools,Karaoke,Pediatricians,Service Stations,Transmission Repair,Pet Training,Electricians,Waffles,Basque,Bridal
0,WKMJwqnfZKsAae75RMP6jA,0.378506,-0.542541,-1.664101,3.333521,1.442221,4.736537,1.883752,6.489992,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
1,9OG5YkX1g2GReZM0AskizA,-1.746647,0.375948,0.600925,-0.299983,-0.693375,-0.211125,-0.530855,-0.154083,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
2,7clCBgNbd-x2Wj96lZ6Mjw,1.086890,-0.238426,0.600925,-0.299983,1.442221,-0.211125,-0.530855,-0.154083,-1.363604,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
3,ppFCk9aQkM338Rgwpl2F5A,-1.038263,-0.493391,0.600925,3.333521,1.442221,-0.211125,-0.530855,-0.154083,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
4,IDtLPgUrqorrpqSLdfMhZQ,0.378506,0.529542,0.600925,3.333521,1.442221,-0.211125,-0.530855,-0.154083,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Zi-F-YvyVOK0k5QD7lrLOg,0.378506,5.364666,0.600925,-0.299983,-0.693375,-0.211125,-0.530855,-0.154083,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
1074,L62NXZYNN0A2dWZvCj0RkQ,1.795275,-0.570188,0.600925,-0.299983,-0.693375,-0.211125,-0.530855,-0.154083,-1.363604,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
1075,uyQGW7kZKk90vEwGo8A7mA,-1.038263,-0.118623,0.600925,-0.299983,-0.693375,-0.211125,-0.530855,-0.154083,-1.363604,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471
1076,Oun4NN-u5yiHIxDqtJnxgA,0.378506,0.962675,0.600925,3.333521,1.442221,-0.211125,-0.530855,-0.154083,0.733351,...,-0.030471,-0.043113,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471,-0.030471


In [ ]:
rating_scaler = MinMaxScaler()
ratings_normalized = rating_scaler.fit_transform(reviews['stars'].values.reshape(-1, 1))

# Assign the normalized values back to the 'stars' column
reviews["stars"] = ratings_normalized

In [ ]:
import networkx as nx

In [ ]:
nx.__version__

'3.4.2'

In [ ]:
torch.__version__

'2.5.1+cu124'

In [ ]:
def create_interaction_graph(users_encoded, destinations_encoded, reviews, user_exclude_columns, business_exclude_columns):
    # Feature columns for users and businesses
    user_feature_columns = [col for col in users_encoded.columns if col not in user_exclude_columns]
    business_feature_columns = [col for col in destinations_encoded.columns if col not in business_exclude_columns]

    # Preprocess user features
    user_features = users_encoded[user_feature_columns].copy()

    # Preprocess business features
    business_features = destinations_encoded[business_feature_columns].copy()

    # Map user_id and business_id to numeric indices
    user_id_mapping = {user_id: idx for idx, user_id in enumerate(users_encoded['user_id'].unique())}
    business_id_mapping = {business_id: idx + len(user_id_mapping) for idx, business_id in enumerate(destinations_encoded['business_id'].unique())}

    # Create the graph
    G = nx.Graph()

    # Add user nodes with preprocessed features
    for _, row in users_encoded.iterrows():
        user_id = row['user_id']
        user_idx = user_id_mapping[user_id]  # Use the numeric index for user
        features = torch.tensor(user_features.loc[row.name].values, dtype=torch.float)
        G.add_node(user_idx, type='user', features=features)

    # Add business nodes with preprocessed features
    for _, row in destinations_encoded.iterrows():
        business_id = row['business_id']
        business_idx = business_id_mapping[business_id]  # Use the numeric index for business
        features = torch.tensor(business_features.loc[row.name].values, dtype=torch.float)
        G.add_node(business_idx, type='business', features=features)

    # Add edges between user and business nodes, including rating (stars)
    for _, interaction in reviews.iterrows():
        user_id = interaction['user_id']
        business_id = interaction['business_id']
        user_idx = user_id_mapping[user_id]
        business_idx = business_id_mapping[business_id]

        # Extract the star rating (assuming it's in a 'stars' column in the reviews dataframe)
        rating = interaction['stars']

        # Add edge with 'rating' as an edge attribute
        G.add_edge(user_idx, business_idx, type='interaction', rating=rating)

    return G

# Example usage:
user_exclude_columns = ['user_id']
business_exclude_columns = ['business_id']

G = create_interaction_graph(users_encoded, destinations_encoded, reviews, user_exclude_columns, business_exclude_columns)


In [ ]:
import torch_geometric
torch_geometric.__version__

'2.6.1'

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:


# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels * 2, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Apply the final linear layer on the concatenated node features
        edge_pred = self.fc(torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=1))
        return edge_pred.squeeze()


In [ ]:
def split_edges(edge_index, split_ratio=[0.7, 0.15, 0.15]):
    num_edges = edge_index.shape[1]
    indices = list(range(num_edges))
    random.shuffle(indices)

    train_size = int(split_ratio[0] * num_edges)
    val_size = int(split_ratio[1] * num_edges)

    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]

    train_edge_index = edge_index[:, train_indices]
    val_edge_index = edge_index[:, val_indices]
    test_edge_index = edge_index[:, test_indices]

    return train_edge_index, val_edge_index, test_edge_index


In [ ]:
# Convert graph edges into edge_index format
edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()

# Split the edges into train, validation, and test sets
train_edge_index, val_edge_index, test_edge_index = split_edges(edge_index)
# Convert the edge attributes (ratings) for training, validation, and testing
train_edge_attr = torch.tensor([G[int(u)][int(v)]['rating'] for u, v in zip(train_edge_index[0].tolist(), train_edge_index[1].tolist())], dtype=torch.float)
val_edge_attr = torch.tensor([G[int(u)][int(v)]['rating'] for u, v in zip(val_edge_index[0].tolist(), val_edge_index[1].tolist())], dtype=torch.float)
test_edge_attr = torch.tensor([G[int(u)][int(v)]['rating'] for u, v in zip(test_edge_index[0].tolist(), test_edge_index[1].tolist())], dtype=torch.float)

# Edge Index (user-item interactions)
# edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()  # Shape: [2, num_edges]

# Separate user and item features
# Check the feature format and make sure it's properly converted
user_features = torch.stack([torch.tensor(node_data['features'], dtype=torch.float) for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'user'])
item_features = torch.stack([torch.tensor(node_data['features'], dtype=torch.float) for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'business'])
user_feature_sizes = [node_data['features'].shape for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'user']
item_feature_sizes = [node_data['features'].shape for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'business']

print(f"User feature sizes: {user_feature_sizes}")
print(f"Item feature sizes: {item_feature_sizes}")
# Combine user and item features into one tensor (assuming they are separate in your graph)
# Example of padding features to the max feature size (if needed)
max_user_feature_size = max(user_feature_sizes, key=lambda x: x[0])[0]
max_item_feature_size = max(item_feature_sizes, key=lambda x: x[0])[0]
max_feature_size = max(max_user_feature_size, max_item_feature_size)

# Padding user features
user_features = torch.stack([
    F.pad(torch.tensor(node_data['features'], dtype=torch.float), (0, max_feature_size - node_data['features'].shape[0]))
    for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'user'
])

# Padding item features
item_features = torch.stack([
    F.pad(torch.tensor(node_data['features'], dtype=torch.float), (0, max_feature_size - node_data['features'].shape[0]))
    for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'business'
])

# Combine user and item features into one tensor
node_features = torch.cat([user_features, item_features], dim=0)

# Create Data objects for train, validation, and test sets
train_data = Data(edge_index=train_edge_index, edge_attr=train_edge_attr, x=node_features)
val_data = Data(edge_index=val_edge_index, edge_attr=val_edge_attr, x=node_features)
test_data = Data(edge_index=test_edge_index, edge_attr=test_edge_attr, x=node_features)

<ipython-input-27-9a55a1dae8fa>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_features = torch.stack([torch.tensor(node_data['features'], dtype=torch.float) for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'user'])
<ipython-input-27-9a55a1dae8fa>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item_features = torch.stack([torch.tensor(node_data['features'], dtype=torch.float) for node_id, node_data in G.nodes(data=True) if node_data['type'] == 'business'])
<ipython-input-27-9a55a1dae8fa>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than 

User feature sizes: [torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.Size([17]), torch.S

In [ ]:
# Initialize the model (e.g., GCN)
model = GCN(in_channels=node_features.size(1), hidden_channels=16, out_channels=1)

# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(train_data)  # Train using the training set
    loss = criterion(out, train_data.edge_attr.view(-1, 1))
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Evaluate the model



<ipython-input-27-9a55a1dae8fa>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  F.pad(torch.tensor(node_data['features'], dtype=torch.float), (0, max_feature_size - node_data['features'].shape[0]))
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([3310, 1])) that is different to the input size (torch.Size([3310])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.836296558380127
Epoch 2, Loss: 0.8178838491439819
Epoch 3, Loss: 0.8005896210670471
Epoch 4, Loss: 0.7845122814178467
Epoch 5, Loss: 0.7697243690490723
Epoch 6, Loss: 0.7558801174163818
Epoch 7, Loss: 0.7426937222480774
Epoch 8, Loss: 0.7300568222999573
Epoch 9, Loss: 0.7179789543151855
Epoch 10, Loss: 0.7063469886779785
Epoch 11, Loss: 0.6951718926429749
Epoch 12, Loss: 0.6844010353088379
Epoch 13, Loss: 0.6737669110298157
Epoch 14, Loss: 0.6633427143096924
Epoch 15, Loss: 0.6531534194946289
Epoch 16, Loss: 0.6430695056915283
Epoch 17, Loss: 0.6330426931381226
Epoch 18, Loss: 0.6230174899101257
Epoch 19, Loss: 0.6129180788993835
Epoch 20, Loss: 0.6027708649635315
Epoch 21, Loss: 0.5925397872924805
Epoch 22, Loss: 0.58219313621521
Epoch 23, Loss: 0.5717372894287109
Epoch 24, Loss: 0.5611971616744995
Epoch 25, Loss: 0.550571858882904
Epoch 26, Loss: 0.5398356914520264
Epoch 27, Loss: 0.5289766192436218
Epoch 28, Loss: 0.5179876685142517
Epoch 29, Loss: 0.506871223449707

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set model to evaluation mode
model.eval()

# No gradient computation for evaluation
with torch.no_grad():
    val_out = model(val_data)
    test_out = model(test_data)
    train_out = model(train_data)

# Calculate evaluation metrics for validation set
val_rmse = mean_squared_error(val_edge_attr.numpy(), val_out.numpy())
val_mae = mean_absolute_error(val_edge_attr.numpy(), val_out.numpy())
val_r2 = r2_score(val_edge_attr.numpy(), val_out.numpy())

# Calculate evaluation metrics for test set
test_rmse = mean_squared_error(test_edge_attr.numpy(), test_out.numpy())
test_mae = mean_absolute_error(test_edge_attr.numpy(), test_out.numpy())
test_r2 = r2_score(test_edge_attr.numpy(), test_out.numpy())

# Calculate evaluation metrics for train set
train_rmse = mean_squared_error(train_edge_attr.numpy(), train_out.numpy())
train_mae = mean_absolute_error(train_edge_attr.numpy(), train_out.numpy())
train_r2 = r2_score(train_edge_attr.numpy(), train_out.numpy())

# Print evaluation metrics
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation R2: {val_r2:.4f}")

print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test R2: {test_r2:.4f}")

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}")
print(f"Train R2: {train_r2:.4f}")


Validation RMSE: 0.1700
Validation MAE: 0.3192
Validation R2: -0.6979
Test RMSE: 0.1599
Test MAE: 0.3144
Test R2: -0.5333
Train RMSE: 0.1154
Train MAE: 0.2808
Train R2: -0.1219


In [ ]:
import numpy as np
from scipy.stats import rankdata

def mean_reciprocal_rank(y_true, y_pred):
    """
    Compute the Mean Reciprocal Rank (MRR).
    """
    ranks = np.argsort(np.argsort(-y_pred)) + 1  # Get ranks (1-based)
    reciprocal_ranks = 1 / ranks[y_true > 0]  # Reciprocal of ranks for relevant items
    return np.mean(reciprocal_ranks) if len(reciprocal_ranks) > 0 else 0.0

def ndcg_at_k(y_true, y_pred, k=10):
    """
    Compute Normalized Discounted Cumulative Gain (NDCG) at K.
    """
    ideal_order = np.argsort(-y_true)  # Best possible ranking
    ideal_dcg = np.sum(y_true[ideal_order][:k] / np.log2(np.arange(2, k + 2)))

    sorted_order = np.argsort(-y_pred)  # Model-predicted ranking
    dcg = np.sum(y_true[sorted_order][:k] / np.log2(np.arange(2, k + 2)))

    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0

def hit_rate_at_k(y_true, y_pred, k=10):
    """
    Compute Hit Rate (HR) at K.
    """
    top_k = np.argsort(-y_pred)[:k]  # Top-K predicted indices
    return int(np.any(y_true[top_k] > 0))  # Check if any relevant item is in top-K

# Convert tensors to NumPy arrays
y_true_val = val_edge_attr.numpy()
y_pred_val = val_out.numpy()

y_true_test = test_edge_attr.numpy()
y_pred_test = test_out.numpy()

y_true_train = train_edge_attr.numpy()
y_pred_train = train_out.numpy()

# Compute ranking metrics for validation, test, and train sets
val_mrr = mean_reciprocal_rank(y_true_val, y_pred_val)
val_ndcg = ndcg_at_k(y_true_val, y_pred_val, k=10)
val_hr = hit_rate_at_k(y_true_val, y_pred_val, k=10)

test_mrr = mean_reciprocal_rank(y_true_test, y_pred_test)
test_ndcg = ndcg_at_k(y_true_test, y_pred_test, k=10)
test_hr = hit_rate_at_k(y_true_test, y_pred_test, k=10)

train_mrr = mean_reciprocal_rank(y_true_train, y_pred_train)
train_ndcg = ndcg_at_k(y_true_train, y_pred_train, k=10)
train_hr = hit_rate_at_k(y_true_train, y_pred_train, k=10)

# Print ranking metrics
print(f"Validation MRR: {val_mrr:.4f}")
print(f"Validation NDCG@10: {val_ndcg:.4f}")
print(f"Validation HR@10: {val_hr:.4f}")

print(f"Test MRR: {test_mrr:.4f}")
print(f"Test NDCG@10: {test_ndcg:.4f}")
print(f"Test HR@10: {test_hr:.4f}")

print(f"Train MRR: {train_mrr:.4f}")
print(f"Train NDCG@10: {train_ndcg:.4f}")
print(f"Train HR@10: {train_hr:.4f}")


Validation MRR: 0.0103
Validation NDCG@10: 0.9286
Validation HR@10: 1.0000
Test MRR: 0.0100
Test NDCG@10: 0.7787
Test HR@10: 1.0000
Train MRR: 0.0027
Train NDCG@10: 0.9111
Train HR@10: 1.0000
